In [18]:
import warnings


warnings.filterwarnings("ignore")

import nltk
from nltk import FreqDist
nltk.download('stopwords')

# !pip install pyLDAvis

import pandas as pd
# pd.set_option("display.max_colwidth", 200)
import numpy as np
import json
import re
import gzip
import spacy

import gensim
from gensim import corpora

import pyLDAvis
# import pyLDAvis.gensim
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from gensim.models.coherencemodel import CoherenceModel

from nltk.corpus import stopwords

import nltk

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer

from nltk.stem import WordNetLemmatizer

from tqdm import tqdm

import gensim
from gensim import corpora

import pyLDAvis
# import pyLDAvis.gensim
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tanma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
import json

In [19]:
from gensim.test.utils import datapath

## Loading in the preprocessed data

In [20]:
engaged_df = pd.read_csv("./data/engaged_df.csv")

In [21]:
engaged_df.head()

,book_id,user_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments,clean_reviews
0,5805,4cbecbc15af3db041a8e0f594c642bb5,58f2301bd2d4bbfc1b51e4e5fb161cfe,5,"Remember, remember, the fifth of November. Thi...",Wed Jun 13 17:55:53 -0700 2012,Mon Jan 30 05:58:01 -0800 2017,Thu Jun 14 00:00:00 -0700 2012,NaN,7,0,"['remember', 'remember', 'fifth', 'part', 'quo..."
1,5805,49cc59f1c479d698507627b401d47ecf,761a17f52538341a085b629a316204a1,4,Tinha apontado este livro como um dos que tinh...,Sun Jan 05 03:27:10 -0800 2014,Tue Oct 28 15:58:20 -0700 2014,Tue Oct 28 15:58:20 -0700 2014,NaN,1,0,"['tinha', 'visto', 'portman', 'hora', 'livro',..."
2,5805,5f03864c758bfceb6d7d5e93eeb20044,ac54c3ce0c9f660c03881b0668f79c60,5,Review coming soon! www.youtube.com/ReadTomes,Tue Feb 12 11:38:36 -0800 2013,Tue Feb 12 12:00:47 -0800 2013,Tue Feb 12 12:00:47 -0800 2013,Tue Feb 12 00:00:00 -0800 2013,1,0,"['review', 'ing', 'soon', 'readtome']"
3,5805,c309dff1695ed8558b29ea8dcd7479b8,0da0bcc469c2acd15350f9a8f0a74e2b,5,"What better way to celebrate Guy Fawkes Day, t...",Tue May 20 09:52:36 -0700 2014,Thu Mar 16 09:49:11 -0700 2017,Wed Nov 05 14:22:10 -0800 2014,Wed Nov 05 00:00:00 -0800 2014,2,0,"['well', 'way', 'celebrate', 'guy', 'day', 're..."
4,5805,0f6b8c04f811e05c8978bd6b66ce7685,7d670b6c8cac0c086e21ae2f1af6eccb,4,To note - I am writing this review a quarter c...,Tue Jul 03 16:43:26 -0700 2012,Fri Feb 27 14:46:00 -0800 2015,Tue Jan 01 00:00:00 -0800 1991,NaN,10,0,"['note', 'write', 'review', 'quarter', 'centur..."


In [41]:
def to_list(x):
    x = x.strip('][').split(', ')
#     print(x)
    x = [i.strip("'") for i in x]
    return x

In [44]:
engaged_df['clean_reviews'] = engaged_df['clean_reviews'].apply(lambda x: to_list(x))

## LDA Training over the reviews

In [50]:
dictionary = corpora.Dictionary(engaged_df['clean_reviews'])

In [51]:
doc_term_matrix = tqdm([dictionary.doc2bow(rev) for rev in engaged_df['clean_reviews']])


  0%|                                                                                        | 0/39787 [01:19<?, ?it/s]


In [52]:
## Setting the Hyperparameter Range

# Topics range
min_topics = 2
max_topics = 9
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')


In [53]:
model_results = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

In [54]:
## Using the coherence metric to tune the LDA model
def compute_coherence_values(corpus, dictionary, k, a, b, test_sample):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=test_sample, dictionary=dictionary, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [55]:
len(engaged_df['clean_reviews'])

39787

In [108]:
for k in tqdm(topics_range, ):
    # iterate through alpha values
    for a in tqdm(alpha):
        # iterare through beta values
        for b in tqdm(beta):
            test_sample = np.random.choice(engaged_df['clean_reviews'], size = 7500)
            cv = compute_coherence_values(corpus=doc_term_matrix, 
                                           dictionary=dictionary, 
                                           k=k, 
                                           a=a, 
                                           b=b,
                                           test_sample = test_sample)
                
            model_results['Topics'].append(k)
            model_results['Alpha'].append(a)
            model_results['Beta'].append(b)
            model_results['Coherence'].append(cv)
            
pd.DataFrame(model_results).to_csv('lda_reviews_tuning_results.csv', index=False)

  0%|                                                                                            | 0/6 [00:00<?, ?it/s]

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

 20%|████████████████▌                                                                  | 1/5 [02:10<08:42, 130.75s/it]

 40%|█████████████████████████████████▏                                                 | 2/5 [04:17<06:25, 128.55s/it]

 60%|█████████████████████████████████████████████████▊                                 | 3/5 [06:43<04:33, 136.56s/it]

 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [09:17<02:23, 143.38s/it]

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [11:47<00:00, 141.43s/it]

 17%|█████████████▊                                                                     | 1/6 [11:47<58:55, 707.14s/it]

  0%|                           

 60%|█████████████████████████████████████████████████▊                                 | 3/5 [07:07<04:44, 142.43s/it]

 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [09:27<02:21, 141.40s/it]

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [11:47<00:00, 141.42s/it]

  0%|                                                                                            | 0/6 [00:00<?, ?it/s]

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

 20%|████████████████▌                                                                  | 1/5 [02:23<09:34, 143.63s/it]

 40%|█████████████████████████████████▏                                                 | 2/5 [04:45<07:07, 142.54s/it]

 60%|█████████████████████████████████████████████████▊                                 | 3/5 [07:06<04:44, 142.09s/it]

 80%|███████████████████████████

 83%|███████████████████████████████████████████████████████████████████▌             | 5/6 [1:04:30<12:54, 774.86s/it]

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

 20%|████████████████▌                                                                  | 1/5 [02:37<10:31, 157.92s/it]

 40%|█████████████████████████████████▏                                                 | 2/5 [05:08<07:41, 153.89s/it]

 60%|█████████████████████████████████████████████████▊                                 | 3/5 [07:42<05:06, 153.50s/it]

 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [10:16<02:33, 153.84s/it]

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [12:54<00:00, 154.86s/it]

  0%|                                                                                            | 0/6 [00:00<?, ?it/s]

  0%|                           

## Train and Save Best Model

In [ ]:
res = pd.DataFrame(model_results)

In [ ]:
res.sort_values(by = "coherence_score")

In [ ]:
lda_model = gensim.models.LdaMulticore(corpus=doc_term_matrix,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)

In [ ]:
lda_model.save(datapath('./data/best_review_model'))

## LDA Training Over The Descriptions

In [10]:
book_df = pd.read_csv('./data/book_p.csv')

In [48]:
book_df['processed_description'] = book_df['processed_description'].apply(lambda x: to_list(x))

In [49]:
book_df.head()

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series,processed_description
0,1401207928,3400,['631559'],US,eng,"[{'count': '42534', 'name': 'to-read'}, {'coun...",NaN,False,4.25,B0064W65UM,...,NaN,2005.0,https://www.goodreads.com/book/show/5805.V_for...,https://images.gr-assets.com/books/1343668985m...,5805,212464,392838,V for Vendetta,V for Vendetta,"[remember, remember, fifth, frightening, power..."
1,394541553,4402,['186595'],US,eng,"[{'count': '92915', 'name': 'to-read'}, {'coun...",NaN,False,4.35,NaN,...,NaN,1991.0,https://www.goodreads.com/book/show/15196.Maus_I,https://images.gr-assets.com/books/1327884972m...,15196,189288,1947012,Maus I: A Survivor's Tale: My Father Bleeds Hi...,Maus I: A Survivor's Tale: My Father Bleeds Hi...,"[story, jewish, survivor, son, cartoonist, try..."
2,1401207529,1790,"['482441', '300099', '194163', '636672']",US,eng,"[{'count': '7924', 'name': 'to-read'}, {'count...",NaN,False,4.22,B0064W65SO,...,NaN,2005.0,https://www.goodreads.com/book/show/59980.Batman,https://images.gr-assets.com/books/1327940389m...,59980,154013,2501570,Batman: Year One,Batman: Year One,"[lieutenant, take, new, post, crime, ride, cor..."
3,1421501686,2399,['205782'],US,eng,"[{'count': '9727', 'name': 'to-read'}, {'count...",NaN,False,4.42,NaN,...,NaN,2005.0,https://www.goodreads.com/book/show/13615.Deat...,https://images.gr-assets.com/books/1419952134m...,13615,142755,1782155,"Death Note, Vol. 1: Boredom (Death Note, #1)","Death Note, Vol. 1: Boredom (Death Note, #1)","[light, ace, student, great, prospect, bore, m..."
4,1607066017,8700,['736247'],US,eng,"[{'count': '78762', 'name': 'to-read'}, {'coun...",NaN,False,4.24,B015XEABR4,...,NaN,2012.0,https://www.goodreads.com/book/show/15704307-s...,https://images.gr-assets.com/books/1486028947m...,15704307,142640,19113524,"Saga, Vol. 1 (Saga, #1)","Saga, Vol. 1 (Saga, #1)","[two, soldier, opposite, side, never, end, gal..."


In [45]:
dictionary = corpora.Dictionary(book_df['processed_description'])

In [46]:
doc_term_matrix = tqdm([dictionary.doc2bow(rev) for rev in book_df['processed_description']])

  0%|                                                                                        | 0/39787 [00:00<?, ?it/s]

In [11]:
## Setting the Hyperparameter Range

# Topics range
min_topics = 2
max_topics = 9
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')


In [12]:
model_results = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

In [108]:
for k in tqdm(topics_range, ):
    # iterate through alpha values
    for a in tqdm(alpha):
        # iterare through beta values
        for b in tqdm(beta):
            test_sample = np.random.choice(book_df['processed_description'], size = 7500)
            cv = compute_coherence_values(corpus=doc_term_matrix, 
                                           dictionary=dictionary, 
                                           k=k, 
                                           a=a, 
                                           b=b,
                                           test_sample = test_sample)
                
            model_results['Topics'].append(k)
            model_results['Alpha'].append(a)
            model_results['Beta'].append(b)
            model_results['Coherence'].append(cv)
            
pd.DataFrame(model_results).to_csv('lda_description_tuning_results.csv', index=False)

  0%|                                                                                            | 0/6 [00:00<?, ?it/s]

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

 20%|████████████████▌                                                                  | 1/5 [02:10<08:42, 130.75s/it]

 40%|█████████████████████████████████▏                                                 | 2/5 [04:17<06:25, 128.55s/it]

 60%|█████████████████████████████████████████████████▊                                 | 3/5 [06:43<04:33, 136.56s/it]

 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [09:17<02:23, 143.38s/it]

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [11:47<00:00, 141.43s/it]

 17%|█████████████▊                                                                     | 1/6 [11:47<58:55, 707.14s/it]

  0%|                           

 60%|█████████████████████████████████████████████████▊                                 | 3/5 [07:07<04:44, 142.43s/it]

 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [09:27<02:21, 141.40s/it]

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [11:47<00:00, 141.42s/it]

  0%|                                                                                            | 0/6 [00:00<?, ?it/s]

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

 20%|████████████████▌                                                                  | 1/5 [02:23<09:34, 143.63s/it]

 40%|█████████████████████████████████▏                                                 | 2/5 [04:45<07:07, 142.54s/it]

 60%|█████████████████████████████████████████████████▊                                 | 3/5 [07:06<04:44, 142.09s/it]

 80%|███████████████████████████

 83%|███████████████████████████████████████████████████████████████████▌             | 5/6 [1:04:30<12:54, 774.86s/it]

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

 20%|████████████████▌                                                                  | 1/5 [02:37<10:31, 157.92s/it]

 40%|█████████████████████████████████▏                                                 | 2/5 [05:08<07:41, 153.89s/it]

 60%|█████████████████████████████████████████████████▊                                 | 3/5 [07:42<05:06, 153.50s/it]

 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [10:16<02:33, 153.84s/it]

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [12:54<00:00, 154.86s/it]

  0%|                                                                                            | 0/6 [00:00<?, ?it/s]

  0%|                           

In [ ]:
res = pd.DataFrame(model_results)

In [ ]:
res.sort_values(by = "coherence_score")

In [ ]:
lda_model = gensim.models.LdaMulticore(corpus=doc_term_matrix,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)

In [ ]:
lda_model.save(datapath('./data/best_description_model'))